# Debug

## Definitions

In [1]:
# impose modules
from sympy import symbols, cos, sin, pi, simplify, atan2, sqrt
from sympy.matrices import Matrix, matrix_multiply_elementwise
import numpy as np
import tf

In [2]:
# define symbols
q1, q2, q3, q4, q5, q6 = symbols('q1:7')

In [3]:
# alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')
# a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
# d1, d2, d3, d4, d5, d6, dG = symbols('d1:8')
# theta1, theta2, theta3, theta4, theta5, theta6, thetaG = symbols('theta1:8')
# q1, q2, q3, q4, q5, q6 = symbols('q1:7')

In [4]:
# identity matrix
I = Matrix([[1, 0, 0], 
            [0, 1, 0], 
            [0, 0, 1]])

In [5]:
# orientation between frame G and frame gripper_link
rGg = Matrix([[0, 0, 1],
              [0, -1, 0],
              [1, 0, 0]])

In [6]:
# origin of frame
origin = Matrix([[0,0,0,1]]).T

In [7]:
# rotation matrices

def rot_x(q):
    R_x = Matrix([[ 1,              0,        0],
                  [ 0,         cos(q),  -sin(q)],
                  [ 0,         sin(q),  cos(q)]])
    
    return R_x
    
def rot_y(q):              
    R_y = Matrix([[ cos(q),        0,  sin(q)],
                  [      0,        1,       0],
                  [-sin(q),        0, cos(q)]])
    
    return R_y

def rot_z(q):    
    R_z = Matrix([[ cos(q),  -sin(q),       0],
                  [ sin(q),   cos(q),       0],
                  [      0,        0,       1]])
    
    return R_z

In [8]:
# translation vectors

def p_x(d):
    t = Matrix([[d], [0], [0]])
    return t

def p_y(d):
    t = Matrix([[0], [d], [0]])
    return t

def p_z(d):
    t = Matrix([[0], [0], [d]])
    return t

In [9]:
# homogeneous transform: put rotation and translation together
def H(R, p=Matrix([[0], [0], [0]])):
    '''
    R is rotation matrix
    p is translation vector
    '''
    D = R.row_join(p)
    D = D.col_join(Matrix([[0,0,0,1]]))
    return D

In [10]:
# homogeneous transform between neighbouring frames
def T(alpha, a, theta, d):
    T = H(rot_x(alpha)) * H(I, p_x(a)) * H(rot_z(theta)) * H(I, p_z(d))
    return T

In [11]:
# DH parameter
s = {'alpha0': 0,     'a0': 0,      'd1': 0.75,  'theta1': q1,
     'alpha1': -pi/2, 'a1': 0.35,   'd2': 0,     'theta2': q2 - pi/2,
     'alpha2': 0,     'a2': 1.25,   'd3': 0,     'theta3': q3,
     'alpha3': -pi/2, 'a3': -0.054, 'd4': 1.5,   'theta4': q4,
     'alpha4': pi/2,  'a4': 0,      'd5': 0,     'theta5': q5,
     'alpha5': -pi/2, 'a5': 0,      'd6': 0,     'theta6': q6,
     'alpha6': 0,     'a6': 0,      'dG': 0.303, 'thetaG': 0}

In [12]:
# Homogeneous transform between frames
T01 = T(s['alpha0'], s['a0'], s['theta1'], s['d1'])
T12 = T(s['alpha1'], s['a1'], s['theta2'], s['d2'])
T23 = T(s['alpha2'], s['a2'], s['theta3'], s['d3'])
T34 = T(s['alpha3'], s['a3'], s['theta4'], s['d4'])
T45 = T(s['alpha4'], s['a4'], s['theta5'], s['d5'])
T56 = T(s['alpha5'], s['a5'], s['theta6'], s['d6'])
T6G = T(s['alpha6'], s['a6'], s['thetaG'], s['dG'])
TGg = H(rGg)

In [13]:
# transform from base_point to G
T0G = T01 * T12 * T23 * T34 * T45 * T56 * T6G

In [14]:
# transform from base_point to G
T0g = T01 * T12 * T23 * T34 * T45 * T56 * T6G * TGg

In [15]:
# initial state of joints
state_i = {q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0}

In [16]:
# origin of frame
origin = Matrix([[0,0,0,1]]).T

## Forward Kinematics

In [17]:
# frame G origin
G = origin

In [18]:
# forward kinematics result
p0g = T0g * G

In [19]:
def FK_check(state, pg, og):
    T0g_evl = np.array(T0g.evalf(subs=state)).astype(np.float32)
    T_0g = tf.transformations.quaternion_matrix(og)
    T_0g[:3,3] = pg
    
    print('T0g read by system:\n')
    print(T0g_evl)
    print('\n T0g from FK:\n')
    print(T_0g)
    print('\n difference ratio:\n')
    print((T0g_evl - T_0g) / T_0g)

In [20]:
# random joint state 1
state1 = {q1: 0.31, q2: 0.83, q3: 1.04, q4: -4.25, q5: 1.45, q6: 5.55}

# random gripper position 1
pg1 = [0.76975, 0.52631, 0.104645]
og1 = [0.699816, 0.667646, 0.0606566, 0.246631]

In [21]:
FK_check(state1, pg1, og1)

T0g read by system:

[[ 0.0981307   0.90351069  0.41717955  0.77129132]
 [ 0.96437132  0.01715435 -0.26399562  0.52974582]
 [-0.24567932  0.42822206 -0.86963642  0.1017136 ]
 [ 0.          0.          0.          1.        ]]

 T0g from FK:

[[ 0.10113891  0.90453939  0.41422145  0.76975   ]
 [ 0.9643786   0.01315638 -0.26419845  0.52631   ]
 [-0.24442756  0.42618705 -0.87098781  0.104645  ]
 [ 0.          0.          0.          1.        ]]

 difference ratio:

[[ -2.97433310e-02  -1.13726668e-03   7.14136373e-03   2.00235863e-03]
 [ -7.54782561e-06   3.03881192e-01  -7.67724968e-04   6.52812446e-03]
 [  5.12118960e-03   4.77493376e-03  -1.55156798e-03  -2.80128265e-02]
 [             nan              nan              nan   0.00000000e+00]]


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in divide
  # This is added back by InteractiveShellApp.init_path()


## Inverse Kinematics

In [22]:
def pose_wc(p_g, o_g):
    '''
    get the orientation and position of wrist center (WC) on frame 6
    inputs:
    p_g: numpy array, [x, y, z] position of gripper to origin
    o_g: numpy array, [x, y, z, w] quaternion of gripper to origin
    '''
    # get homogeneous transform of rotation from quaternion og
    T_0g = tf.transformations.quaternion_matrix(o_g)
    # identify pg with the translation part
    T_0g[:3, 3] = p_g

    # np array rGg
    r_Gg = np.array(rGg).astype(np.float32)
    # r06: the 3 rotation from frame 0 to frame 6
    r_06 = T_0g[:3, :3].dot(r_Gg)
    # pwc: position of WC
    x = np.array([[1.], [0.], [0.]])
    p_wc = np.array(p_g).reshape(3,1) - s['dG'] * T_0g[:3,:3].dot(x)
    # return T_0g, r_06, p_wc
    return T_0g, r_06, p_wc

In [23]:
T_0g, r_06, p_wc = pose_wc(pg1, og1)
print(T_0g)
print(r_06)
print(p_wc)

[[ 0.10113891  0.90453939  0.41422145  0.76975   ]
 [ 0.9643786   0.01315638 -0.26419845  0.52631   ]
 [-0.24442756  0.42618705 -0.87098781  0.104645  ]
 [ 0.          0.          0.          1.        ]]
[[ 0.41422145 -0.90453939  0.10113891]
 [-0.26419845 -0.01315638  0.9643786 ]
 [-0.87098781 -0.42618705 -0.24442756]]
[[ 0.73910491]
 [ 0.23410328]
 [ 0.17870655]]


In [24]:
def Q_1(p_wc):
    '''
    calculate q1
    inputs:
    p_wc: position of WC to frame 0
    '''
    q_1 = atan2(p_wc[1], p_wc[0]).evalf()
    T_01 = np.array(T(s['alpha0'], s['a0'], s['theta1'], s['d1']).evalf(subs={q1: q_1})).astype(np.float32)
    return q_1, T_01

In [25]:
q_1, T_01 = Q_1(p_wc)

In [26]:
def P_wc1(p_wc):
    '''
    get position of WC to frame 1
    p_wc: position of WC to frame 0
    '''
    _, T_01 = Q_1(p_wc)
    # get inverse matrix of T_01
    T_01_inv = np.linalg.inv(T_01)
    # 4-vector of p_wc
    p_wc_4 = np.append(p_wc, 1).reshape(4, 1)
    # p_wc1: WC position to frame 1
    p_wc1 = T_01_inv.dot(p_wc_4)
    # p_wc1 = p_wc1[:3]
    return p_wc1

In [27]:
p_wc1 = P_wc1(p_wc)
print(p_wc1)

[[  7.75293767e-01]
 [  1.35838910e-09]
 [ -5.71293450e-01]
 [  1.00000000e+00]]


In [28]:
def Q_2(p_wc1):
    '''
    calculate q2
    inputs:
    p_wc1: WC position to frame 1, numpy array shape=(4, 1),
    '''
    k34 = sqrt(s['d4'] ** 2 + s['a3'] ** 2)
    k24 = sqrt((p_wc1[0] - s['a1']) ** 2 + p_wc1[2] ** 2)
    cosq23 = (s['a2']**2 + k24**2 - k34**2) / (2 * s['a2'] * k24)
    q23 = atan2(sqrt(1 - cosq23**2), cosq23)

    q21 = atan2(p_wc1[2], p_wc1[0]-s['a1'])

    q_2 = (pi/2 - q23 - q21).evalf()
    T_12 = np.array(T(s['alpha1'], s['a1'], s['theta2'], s['d2']).evalf(subs={q2: q_2})).astype(np.float32)
    return q_2, T_12

In [29]:
q_2, T_12 = Q_2(p_wc1)

In [30]:
def Q_3(p_wc1):
    '''
    calculate q3
    inputs:
    p_wc1: WC position to frame 1, numpy array shape=(4, 1),
    '''
    q34 = atan2(s['d4'], s['a3'])
    k34 = sqrt(s['d4'] ** 2 + s['a3'] ** 2)
    k24 = sqrt((p_wc1[0] - s['a1']) ** 2 + p_wc1[2] ** 2)

    cosq24 = (s['a2']**2 + k34**2 - k24**2) / (2 * s['a2'] * k34)
    q24 = atan2(sqrt(1 - cosq24**2), cosq24)

    q_3 = (-q24 + q34).evalf()
    T_23 = np.array(T(s['alpha2'], s['a2'], s['theta3'], s['d3']).evalf(subs={q3: q_3})).astype(np.float32)
    return q_3, T_23

In [31]:
q_3, T_23 = Q_3(p_wc1)

In [32]:
T_03 = T_01.dot(T_12).dot(T_23)
r_03 = T_03[:3, :3]

In [33]:
def Q_456(r_03, r_06):
    '''
    calculate q4, q5, q6
    inputs:
    r_03: rotation from frame 3 to frame 0, numpy array, shape=(3,3)
    r_06: rotation from frame 6 to frame 6, numpy array, shape=(3,3)
    '''
    r_03_inv = np.linalg.inv(r_03)
    r_36 = r_03_inv.dot(r_06)

    r21, r22, r23 = r_36[1, :]
    r13, r33 = r_36[[0,2], 2]

    q_4 = atan2(r33, -r13).evalf()
    q_5 = atan2(sqrt(r21**2 + r22**2), r23).evalf()
    q_6 = atan2(-r22, r21).evalf()
    T_34 = np.array(T(s['alpha3'], s['a3'], s['theta4'], s['d4']).evalf(subs={q4: q_4})).astype(np.float32)
    T_45 = np.array(T(s['alpha4'], s['a4'], s['theta5'], s['d5']).evalf(subs={q5: q_5})).astype(np.float32)
    T_56 = np.array(T(s['alpha5'], s['a5'], s['theta6'], s['d6']).evalf(subs={q6: q_6})).astype(np.float32)
    return q_4, q_5, q_6, T_34, T_45, T_56

In [34]:
q_4, q_5, q_6, T_34, T_45, T_56 = Q_456(r_03, r_06)

In [35]:
state_ik = {q1: q_1, q2: q_2, q3: q_3, q4: q_4, q5: q_5, q6: q_6}

In [36]:
FK_check(state_ik, pg1, og1)

T0g read by system:

[[ 0.10113892  0.90453941  0.41422144  0.67384279]
 [ 0.9643786   0.01315637 -0.26419845  0.49593246]
 [-0.24442756  0.42618704 -0.87098783  0.14392622]
 [ 0.          0.          0.          1.        ]]

 T0g from FK:

[[ 0.10113891  0.90453939  0.41422145  0.76975   ]
 [ 0.9643786   0.01315638 -0.26419845  0.52631   ]
 [-0.24442756  0.42618705 -0.87098781  0.104645  ]
 [ 0.          0.          0.          1.        ]]

 difference ratio:

[[  8.40952252e-08   1.55366114e-08  -3.11086777e-08  -1.24595274e-01]
 [ -7.46061191e-09  -7.04428226e-07   1.08258562e-08  -5.77179612e-02]
 [  1.11662783e-08  -1.61275370e-08   2.21682256e-08   3.75375968e-01]
 [             nan              nan              nan   0.00000000e+00]]


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in divide
  # This is added back by InteractiveShellApp.init_path()
